In [1]:
import geopandas as gpd
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Here I am  reading in all of my files after making the clean and it all Tennessee based
res = gpd.read_file('res_refined.geojson')
zipcode = gpd.read_file('zipcode_refined.geojson')
tn = gpd.read_file('tennessee_refined.geojson')
trail = gpd.read_file('trails_refined.geojson')
rail = gpd.read_file('rail_refined.geojson')
park = gpd.read_file('parks_refined.geojson')

### How accessible is outdoor recreation in Tennessee, and where should future trail or park investments be prioritized?

### Can we rank ZIPs for trail expansion priority?

In [15]:
# I am joining trail with zip to see how much trail is in each of the zipcodes
trail_zip = gpd.sjoin(trail, zipcode[['ZCTA5CE20', 'geometry']], how='left', predicate='intersects')
trail_by_zip = trail_zip.groupby('ZCTA5CE20')['SEGLEN'].sum().reset_index().rename(columns={'SEGLEN': 'trail_mileage'})
trail_by_zip.head(1)

,ZCTA5CE20,trail_mileage
0,37013,0.180698


In [16]:
# Here I am joining reservations with zip to find where there are lots of reservations made
res_zip = gpd.sjoin(res, zipcode[['ZCTA5CE20', 'geometry']], how='left', predicate='within')
res_by_zip = res_zip.groupby('ZCTA5CE20').size().reset_index(name='reservation_count')
res_by_zip.head(1)

,ZCTA5CE20,reservation_count
0,37013,5


In [17]:
# Here I am creating a summary by zipcode for analysis
zip_stats = zipcode[['ZCTA5CE20', 'Median Income', 'Total Population']]
zip_summary = res_by_zip.merge(trail_by_zip, on='ZCTA5CE20', how='outer').merge(zip_stats, on='ZCTA5CE20', how='left').fillna(0)

In [18]:
# Here I am scaling my value for easier calculations
scaler = MinMaxScaler()
zip_summary[['norm_trail', 'norm_res', 'norm_income', 'norm_pop']] = scaler.fit_transform(
    zip_summary[['trail_mileage', 'reservation_count', 'Median Income', 'Total Population']]
)

In [19]:
# We have a few calculations that are opposite from the rest, like trail being larger means that 
# there is less need for trails. So we are going to invert these ones
zip_summary['trail_need'] = 1 - zip_summary['norm_trail']
zip_summary['income_priority'] = 1 - zip_summary['norm_income']

In [20]:
# Here we are weighting the scores, so we can prioritize where reservation are hot and 
# where there are few trails
zip_summary['priority_score'] = (zip_summary['trail_need'] * 0.4 + zip_summary['norm_res'] * 0.3 +
    zip_summary['income_priority'] * 0.2 + zip_summary['norm_pop'] * 0.1)

In [21]:
# Here we are just ranking zipcodes by the priority score we just made to see where trails are most
# needed
zip_summary['trail_priority_rank'] = zip_summary['priority_score'].rank(ascending=False)

In [22]:
# Here we are exporting for Tableau
zip_summary.to_csv(exit + 'trail_expansion_priority.csv', index=False)
zip_summary.head(1)

,ZCTA5CE20,reservation_count,trail_mileage,Median Income,Total Population,norm_trail,norm_res,norm_income,norm_pop,trail_need,income_priority,priority_score,trail_priority_rank
0,37013,5.0,0.180698,61801,102184,0.001806,0.022523,0.999886,1.0,0.998194,0.000114,0.506057,6.0


In [23]:
# Here I am adding in some geometry and creating a Geo file for Taleau
zip_geo = zipcode[['ZCTA5CE20', 'geometry']]
zip_summary = zip_summary.merge(zip_geo, on = 'ZCTA5CE20', how = 'inner')
zip_gdf = gpd.GeoDataFrame(zip_summary, geometry='geometry')
zip_gdf.set_crs("EPSG:4326", inplace=True)
zip_gdf.to_file(exit + "trail_expansion_priority.geojson", driver="GeoJSON")

"How does median income vary across zipcodes with high vs. low recreation access scores?"

### Are there rail segments that run near parks or residential areas that are inactive and could be repurposed?

In [ ]:
# Here I am looking for all abandoned rails
inactive_rail = rail[
    rail['railusage_description'].str.lower().str.contains('abandoned|inactive|out of service', na=False)
]

In [ ]:
# Here I am taking a file of all the inactive rails
inactive_rail.to_file('rails_inactive.geojson', driver='GeoJSON')

In [ ]:
# Here I am changing the crs to create a buffer
inactive_rail = inactive_rail.to_crs("EPSG:3857")
park = park.to_crs(inactive_rail.crs)
zipcode = zipcode.to_crs(inactive_rail.crs)

In [ ]:
# Here I am adding in a buffer for the railways
rail_buffer = inactive_rail.copy()
rail_buffer['geometry'] = rail_buffer.buffer(900)

In [ ]:
# Hear I am define inactive rails that are near parks
rail_near_parks = gpd.sjoin(rail_buffer, park[['geometry']], how='inner', predicate='intersects')
rails_with_parks = rail_near_parks['permanent_identifier'].unique()

# Hear I am define inactive rails that are near residential ZIPs (centroid-based)
zipcode['center'] = zipcode.centroid
zip_centers = gpd.GeoDataFrame(geometry=zipcode['center'], crs=zipcode.crs)
rail_near_zip = gpd.sjoin(rail_buffer, zip_centers, how='inner', predicate='intersects')
rails_with_zip = rail_near_zip['permanent_identifier'].unique()

In [ ]:
# Hear I am define inactive rails that are near both parks or ZIPs
rails_candidate_ids = set(rails_with_parks).union(set(rails_with_zip))

# Here I filter original rail GeoDataFrame
rails_to_repurpose = inactive_rail[inactive_rail['permanent_identifier'].isin(rails_candidate_ids)]
rails_to_repurpose = rails_to_repurpose.drop(columns=['center'])
# Here I am saving for visualization
rails_to_repurpose.to_file('rails_to_repurpose.geojson', driver='GeoJSON')

### Where are the largest trail gaps between parks, trails, and residential areas?

In [ ]:
# Here I am defining the crs to allow for buffering
park = park.to_crs("EPSG:3857")
rail = rail.to_crs(park.crs)
trail = trail.to_crs(park.crs)
zipcodes = zipcode.to_crs(park.crs)

In [ ]:
# Here I am adding a buffer to the trails
trail_buffer = trail.copy()
trail_buffer['geometry'] = trail_buffer.buffer(1000)

In [ ]:
#Here I am seeing where there are parks and rails that dont connect with trails
parks_no_trail = park[~park.geometry.intersects(trail_buffer.unary_union)]
rails_no_trail = rail[~rail.geometry.intersects(trail_buffer.unary_union)]

In [ ]:
#Here I am reseting the zip geometry
zip_gdf = gpd.GeoDataFrame(zipcodes[['ZCTA5CE20']], geometry=zipcodes['geometry'], crs=zipcodes.crs)
zips_with_trails = zip_gdf[zip_gdf.geometry.intersects(trail_buffer.unary_union)]

# Here I flag ZIPs that DO NOT intersect any trail buffer = "trail desert ZIPs"
res_zips_no_trail = zip_gdf[~zip_gdf['ZCTA5CE20'].isin(zips_with_trails['ZCTA5CE20'])]

In [ ]:
# Here i am just checking types
print(parks_no_trail.dtypes)
res_zips_no_trail.geom_type

In [ ]:
# Here I am reseting crs and creating data base for visuals
parks_no_trail = parks_no_trail.to_crs("EPSG:4326")
rails_no_trail = rails_no_trail.to_crs(parks_no_trail.crs)
res_zips_no_trail = res_zips_no_trail.to_crs(parks_no_trail.crs)
parks_no_trail = parks_no_trail.set_geometry('geometry')
rails_no_trail = rails_no_trail.set_geometry('geometry')
res_zips_no_trail = res_zips_no_trail.set_geometry('geometry')
parks_no_trail.to_file('parks_with_no_trail_access.geojson', driver='GeoJSON')
rails_no_trail.to_file('rails_with_no_trail_access.geojson', driver='GeoJSON')
res_zips_no_trail.to_file('zips_with_no_trail_access.geojson', driver='GeoJSON')

In [ ]:
# testing
zip_list = res_zips_no_trail['ZCTA5CE20'].dropna().unique()

In [ ]:
# testing
res_zips_no_trail = zipcode[zipcode['ZCTA5CE20'].isin(zip_list)][['ZCTA5CE20', 'geometry']]
res_zips_no_trail = res_zips_no_trail.to_crs("EPSG:4326")

In [ ]:
# testing
res_zips_no_trail.to_file('zips_with_no_trail_access.geojson', driver='GeoJSON')

### Which zipcodes have high public land density but low population density?

In [ ]:
# Here i am allowing for buffers
zipcode = zipcode.to_crs("EPSG:3857")
park = park.to_crs(zipcode.crs)
trail = trail.to_crs(zipcode.crs)

In [ ]:
# Here I am adding buffer to trails
trail_buffer = trail.copy()
trail_buffer['geometry'] = trail_buffer.buffer(50) 

In [ ]:
# Here I am joining park polygons to ZIPs
park_zip = gpd.sjoin(park, zipcode[['ZCTA5CE20', 'geometry']], how='inner', predicate='intersects')
park_zip['park_area'] = park_zip.geometry.area
park_area_by_zip = park_zip.groupby('ZCTA5CE20')['park_area'].sum().reset_index()

In [ ]:
# Here I am looking for zips connected to trails
trail_zip = gpd.sjoin(trail_buffer, zipcode[['ZCTA5CE20', 'geometry']], how='inner', predicate='intersects')
trail_zip['trail_area'] = trail_zip.geometry.area
trail_area_by_zip = trail_zip.groupby('ZCTA5CE20')['trail_area'].sum().reset_index()

In [ ]:
# Starting a dataset with ZIP pop and land area
zip_summary = zipcode[['ZCTA5CE20', 'Total Population', 'ALAND20']].copy()  # ALAND20 = land area in sqm

# MHere I am merging in public land areas
zip_summary = zip_summary.merge(park_area_by_zip, on='ZCTA5CE20', how='left')
zip_summary = zip_summary.merge(trail_area_by_zip, on='ZCTA5CE20', how='left')

# Here I fill nulls (for ZIPs with no parks or trails)
zip_summary[['park_area', 'trail_area']] = zip_summary[['park_area', 'trail_area']].fillna(0)

# Here I am adding total public land area
zip_summary['public_land_area'] = zip_summary['park_area'] + zip_summary['trail_area']

# Here I am calculating public land density (per sq km)
zip_summary['public_land_density'] = zip_summary['public_land_area'] / (zip_summary['ALAND20'] / 1e6)

# Here I am adding population density (people per sq km)
zip_summary['population_density'] = zip_summary['Total Population'] / (zip_summary['ALAND20'] / 1e6)


In [ ]:
# analysis
low_pop_cutoff = zip_summary['population_density'].median()
zip_priority = zip_summary[zip_summary['population_density'] < low_pop_cutoff]

# Here I sort by public land density
zip_priority = zip_priority.sort_values('public_land_density', ascending=False)
zipcode = zipcode.to_crs("EPSG:4326")
zip_priority = zip_priority.merge(zip_geo, on = 'ZCTA5CE20', how = 'inner')
# Here I export for visuals
zip_priority.to_csv('zip_high_land_low_pop.csv', index=False)
zip_priority.to_file('zip_high_land_low_pop.geojson', driver='GeoJSON')

### How much of each zipcode is covered by recreational infrastructure?

In [ ]:
# Here I am allowing for buffering
zipcode = zipcode.to_crs("EPSG:3857")  
park = park.to_crs(zipcode.crs)
trail = trail.to_crs(zipcode.crs)

In [ ]:
# Here I am adding in a buffer to the trails
trail_buffer = trail
trail_buffer['geometry'] = trail_buffer.buffer(100)

In [ ]:
# Here I am combining the geometries
all_recreation = pd.concat([trail_buffer[['geometry']], park[['geometry']]])

# Here I am making a combined polygon for analysis
recreation_union = all_recreation.unary_union

In [ ]:
# Here I am looking to add recreation by zip
zipcode['recreation_area'] = zipcode.geometry.intersection(recreation_union)
zipcode['recreation_area_sqm'] = zipcode['recreation_area'].area

# Here I calcualte total zip area 
zipcode['zip_area_sqm'] = zipcode.geometry.area

# Here I calculate % coverage by recreation
zipcode['recreation_pct'] = zipcode['recreation_area_sqm'] / zipcode['zip_area_sqm'] * 100

In [ ]:
# Here I export for visualization
zipcode_summary = zipcode[['ZCTA5CE20', 'recreation_pct', 'zip_area_sqm', 'recreation_area_sqm']]
zipcode_summary.to_csv('recreation_coverage_by_zip.csv', index=False)

### Are there trail deserts (zipcodes or areas with no trails within 1 mile)?

In [ ]:
# Here I allow for buffers
zipcode = zipcode.to_crs("EPSG:3857")
trail = trail.to_crs(zipcode.crs)

# Here I create a buffer for the trails
trail_buffer_1mi = trail.copy()
trail_buffer_1mi['geometry'] = trail_buffer_1mi.buffer(1609)

In [ ]:
# Here I check to see which zips intersect with trails
zip_with_trails = gpd.sjoin(zipcode, trail_buffer_1mi[['geometry']], how='inner', predicate='intersects')

# Here I store the unique zips with any trail access
zip_codes_with_trails = zip_with_trails['ZCTA5CE20'].unique()

In [ ]:
# Here I define the deserts as the zips that didnt have any access
zipcode['trail_desert'] = ~zipcode['ZCTA5CE20'].isin(zip_codes_with_trails)

In [ ]:
# Here I export for visuals
trail_deserts = zipcode[zipcode['trail_desert']][['ZCTA5CE20', 'geometry']]
trail_deserts.to_file('trail_deserts.geojson', driver='GeoJSON')

# Here I print the no trail zips
print(trail_deserts[['ZCTA5CE20']])

### Which zipcodes have the longest total trail length?

In [ ]:
# Here I combine trails and zips
trail_zip = gpd.sjoin(trail[['SEGLEN', 'geometry']], zipcode[['ZCTA5CE20', 'geometry']], how='left', predicate='intersects')

In [ ]:
# Here I group by zips and sum trail segment lengths
trail_by_zip = trail_zip.groupby('ZCTA5CE20')['SEGLEN'].sum().reset_index().rename(columns={'SEGLEN': 'total_trail_length'})

In [ ]:
# Here I ranks the zips for longest trails
trail_by_zip['trail_rank'] = trail_by_zip['total_trail_length'].rank(ascending=False)

In [ ]:
# Here I export for visuals
trail_by_zip.to_csv('zip_trail_lengths.csv', index=False)

### Do lower-income or rural ZIP codes make fewer reservations?

In [3]:
# Here I am just defining the folder I want to store the files in 
exit = 'tableau/'

In [4]:
# Here I am joining the reservation data with the zipcodes and demographics
res_with_zip = gpd.sjoin(res, zipcode[['ZCTA5CE20', 'Median Income', 'Total Population', 'geometry']], how='left', predicate='within')
res_with_zip.head(1)

,parentlocation,park,sitetype,usetype,inventorytype,facilityzip,customerzip,average_totalpaid,sum_nights,average_nights,sum_numberofpeople,average_numberofpeople,geometry,index_right,ZCTA5CE20,Median Income,Total Population
0,Barkley Lake,DOVER,GROUP SHELTER ELECTRIC,Day,CAMPING,310,37015,35.0,0.0,0.0,30.0,30.0,POINT (-87.84306 36.49028),399,37058,56310,7620


In [5]:
# Here I am doing a group by to get the total reservations and other aggregates by zipcode
reservation_counts = res_with_zip.groupby('ZCTA5CE20').agg({'parentlocation': 'count',                  
    'average_totalpaid': 'mean', 'sum_nights': 'sum', 'average_nights': 'mean',
    'sum_numberofpeople': 'sum', 'average_numberofpeople': 'mean'
}).reset_index().rename(columns={'parentlocation': 'reservation_count'})
reservation_counts.head(1)

,ZCTA5CE20,reservation_count,average_totalpaid,sum_nights,average_nights,sum_numberofpeople,average_numberofpeople
0,37013,5,49.0,0.0,0.0,220.0,44.0


In [6]:
# Here I am merging median income and total population back into reservation in preperation for Tableau
zip_analysis = pd.merge(reservation_counts, zipcode[['ZCTA5CE20', 'Median Income', 'Total Population']], on='ZCTA5CE20', how='left')
zip_analysis.to_csv(exit + 'zip_reservation_income_analysis.csv', index=False)
zip_analysis.head(1)

,ZCTA5CE20,reservation_count,average_totalpaid,sum_nights,average_nights,sum_numberofpeople,average_numberofpeople,Median Income,Total Population
0,37013,5,49.0,0.0,0.0,220.0,44.0,61801,102184


### Do lower-income or rural zipcodes make fewer reservations?

In [ ]:
# Here I am joining the reservation data with the zipcodes and demographics
res_with_zip = gpd.sjoin(res, zipcode[['ZCTA5CE20', 'Median Income', 'Total Population', 'geometry']], how='left', predicate='within')
res_with_zip.head(1)

In [ ]:
# Here I am doing a group by to get the total reservations and other aggregates by zipcode
reservation_counts = res_with_zip.groupby('ZCTA5CE20').agg({'parentlocation': 'count',                  
    'average_totalpaid': 'mean', 'sum_nights': 'sum', 'average_nights': 'mean',
    'sum_numberofpeople': 'sum', 'average_numberofpeople': 'mean'
}).reset_index().rename(columns={'parentlocation': 'reservation_count'})
reservation_counts.head(1)

In [ ]:
# Here I am merging median income and total population back into reservation in preperation for Tableau
zip_analysis = pd.merge(reservation_counts, zipcode[['ZCTA5CE20', 'Median Income', 'Total Population']], on='ZCTA5CE20', how='left')
zip_analysis.to_csv('zip_reservation_income_analysis.csv', index=False)
zip_analysis.head(10)

### Which ZIP codes generate the most outdoor reservations?

In [1]:
# Here I am looking at the count of origin zips to show where people are coming from
origin_zip_counts = res.groupby('customerzip').size().reset_index(name='reservation_count')
# I also wanted to add in some of the other stats
origin_zip_stats = res.groupby('customerzip').agg({'average_totalpaid': 'mean',
    'average_numberofpeople': 'mean', 'sum_nights': 'sum'}).reset_index()
# Here I merge the two together for an origin summary and export for Tableau
zip_origin_summary = pd.merge(origin_zip_counts, origin_zip_stats, on='customerzip', how='left')
zip_origin_summary.to_csv(exit + 'zip_origin_reservations.csv', index=False)

NameError: name 'res' is not defined

### How does Income vary across ZIPs with high vs. low recreation access scores?

In [36]:
# Here I merge trail and reservation data to zipcode and fill in na with 0's
zip = zipcode[['ZCTA5CE20', 'Total Population', 'Median Income]]
access = zip.merge(trail_by_zip, on='ZCTA5CE20', how='left')
access = access.merge(res_by_zip, on='ZCTA5CE20', how='left')
access = access.fillna(0)

In [37]:
# Here I am making a flag for high and low access based on trail mileage
trail_mean = access['trail_mileage'].mean()
access['access_group'] = access['trail_mileage'].apply(
    lambda x: 'High Access' if x >= trail_mean else 'Low Access')

In [38]:
# Here I export for visuals 
access.to_csv(exit + 'simple_access_income.csv', index=False)